### Preprocess logs

In [7]:
# first time to create venv and install packages

# %conda create -n corrid-dev anaconda -y
# %conda activate corrid-dev -y

# %conda install -n corrid-dev pandas -y

# %conda install pip

In [8]:
from sklearn import preprocessing
import numpy
import pandas as pd

def log_pre_process(csv_file_path, memory_reduction, columns_to_drop=[]):
    print(f'log_pre_process: {csv_file_path}')
    data_df = pd.read_csv(csv_file_path, quotechar="'")

    print('Columns before removal:')
    list_of_column_names = list(data_df.columns)
    print(list_of_column_names)

    data_df = data_df.drop(columns=columns_to_drop)

    print('Columns after removal:')
    list_of_column_names = list(data_df.columns)
    print(list_of_column_names)
    print()

    slice_start_col = 0
    slice_end_col = len(data_df.columns)-1

    x = data_df.iloc[:, slice_start_col:slice_end_col]
    y = data_df.iloc[:, slice_end_col:]

    if memory_reduction:
        enc = preprocessing.OrdinalEncoder(dtype=numpy.int8)
        x = enc.fit_transform(x)

    data = (x, y)
    return data


bpic17_logs_with_interventions_path = './data_bpic17_readyToUse_preprocessed_for_adaptation_classification.csv'
bpic17_logs_columns_to_drop = []
bpic17_logs_with_interventions = log_pre_process(bpic17_logs_with_interventions_path, memory_reduction=True, columns_to_drop=bpic17_logs_columns_to_drop)

synthetic_logs_with_adaptations_path = './sample_sequence_simulation_logs_multi_adapted_noisy_encoded.csv'
synthetic_logs_columns_to_drop = ["event:event_name@A","event:event_name@B","event:event_name@C","event:event_name@D","event:event_name@E","event:event_name@F","event:event_name@I","event:event_name@process end event","event:event_name@process start event","event:concept:name@A","event:concept:name@B","event:concept:name@C","event:concept:name@D","event:concept:name@E","event:concept:name@F","event:concept:name@I","event:concept:name@process end event","event:concept:name@process start event","event:adaptation_action@insert","event:adaptation_action@no-action","event:adaptation_action@skip","event:@@index","event:@@case_index","event:trace_id","succession:concept:name@A#B","succession:concept:name@B#C","succession:concept:name@C#D","succession:concept:name@D#E","succession:concept:name@D#F","succession:concept:name@E#F","succession:concept:name@F#process end event","succession:concept:name@process end event#I","succession:concept:name@process start event#A", "event:resource"]
synthetic_logs_with_adaptations = log_pre_process(synthetic_logs_with_adaptations_path, memory_reduction=False, columns_to_drop=synthetic_logs_columns_to_drop)

log_pre_process: ./data_bpic17_readyToUse_preprocessed_for_adaptation_classification.csv
Columns before removal:
['ApplicationType', 'LoanGoal', 'RequestedAmount', 'CreditScore', 'timesincefirstcase', 'duration', 'FirstWithdrawalAmount', 'MonthlyCost', 'NumberOfTerms', 'OfferedAmount', 'open_cases', 'month', 'weekday', 'hour', 'treatment']
Columns after removal:
['ApplicationType', 'LoanGoal', 'RequestedAmount', 'CreditScore', 'timesincefirstcase', 'duration', 'FirstWithdrawalAmount', 'MonthlyCost', 'NumberOfTerms', 'OfferedAmount', 'open_cases', 'month', 'weekday', 'hour', 'treatment']

log_pre_process: ./sample_sequence_simulation_logs_multi_adapted_noisy_encoded.csv
Columns before removal:
['trace_id', 'event:event_name@A', 'event:event_name@B', 'event:event_name@C', 'event:event_name@D', 'event:event_name@E', 'event:event_name@F', 'event:event_name@I', 'event:event_name@process end event', 'event:event_name@process start event', 'event:concept:name@A', 'event:concept:name@B', 'even

In [9]:
%matplotlib inline



### Classifier comparison

A comparison of a several classifiers in scikit-learn 


In [14]:
# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn import tree
from sklearn.tree import export_text
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    # "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=6),
    RandomForestClassifier(max_depth=6, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

datasets = [
    bpic17_logs_with_interventions,
    synthetic_logs_with_adaptations
]

dataset_names = [
    "bpic17_logs_with_interventions",
    "synthetic_logs_with_adaptations"
]

# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, normalize features
    X, y = ds
    X = StandardScaler().fit_transform(X)

    print('---------------------------------------------------------------------')
    print(f'dataset: {dataset_names[ds_cnt]}')

    # iterate over classifiers
    for name, clf in zip(names, classifiers):

        print('------------------------------------------------------')
        print('Classifier:', name)

        scores = cross_val_score(clf, X, y.values.ravel(), cv=10, scoring='f1_weighted')
        print('10-fold cross-validation mean F1:', numpy.mean(scores))


dataset: bpic17_logs_with_interventions
-----------------------------------------------------------
Classifier: Nearest Neighbors
10-fold cross-validation mean F1: 0.6156468709837416
-----------------------------------------------------------
Classifier: Linear SVM
10-fold cross-validation mean F1: 0.5879738165563744
-----------------------------------------------------------
Classifier: RBF SVM
10-fold cross-validation mean F1: 0.3761762134602237
-----------------------------------------------------------
Classifier: Decision Tree
10-fold cross-validation mean F1: 0.7667181488461632
-----------------------------------------------------------
Classifier: Random Forest
10-fold cross-validation mean F1: 0.6622528807500939
-----------------------------------------------------------
Classifier: Neural Net
10-fold cross-validation mean F1: 0.7220517263425028
-----------------------------------------------------------
Classifier: AdaBoost
10-fold cross-validation mean F1: 0.7095708771460894
